## Network Filtering and Processing
---
Run this code block by block to convert a street network in ESRI Shapefile, GeoJSON, or GeoPackage format into a routable network graph to use in BikewaySim. The default network used is OpenStreetMap, but other networks can be provided so long as the network is already in link and node structure.

In [1]:
from bikewaysim.paths import config
from bikewaysim.network import network_filter

## General Settings Dictionary
---
The only input needed for this code is the project directory and the settings dictionary. The settings dictionary has 6 inputs that are commented out below. Note that subsequent code blocks were used for other runs, and should be left commented out so it doesn't overwrite the settings dictionary.

In [2]:
settings = {
    'project_filepath': config['network_fp'], # will create a networks folder in this directory
    'project_crs': config['projected_crs_epsg'], # output CRS that all exported files will be in
    'studyarea_filepath': config['studyarea_fp'], # Specify what area you want to mask the network data by. Only network links that are partially or fully within the study area will be imported. Note: network links are NOT clipped (because this cuts off nodes).
    'studyarea_layer': None, # use if a gpkg or gdb file with multiple layers
    'use_bbox': True # use bounding box of the study area rather than the perimeter
}
settings['studyarea'] = network_filter.import_study_area(settings) #Adds study area geodataframe to settings dictionary

The study area is 1 square miles.


/Users/tannerpassmore/Documents/GitHub/BikewaySimDev/src/bikewaysim/network/network_filter.py:59: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  sqmi = int(studyarea.geometry.area / 5280**2)


## Define Link Types
---
Each link in a network represents a transportation feature, but these features are of various types (local roads, Interstates, bike paths, etc.) and serve different transportation modes (pedestrians, cars, bikes, etc.).

This project is primarily concerned with bicycle routing so there are certain features that we can safely remove from the network such as Interstates and inaccessible, private roads to reduce the size of the network. In addition, we want to classify the type of link to know whether we should add additional restrictions/impedance factors to bike on them (e.g., staircases might be necessary to connect destinations but need to be identified separately so an impedance can be added).

**Imported networks will be projected to CRS defined in the settings dictionary**

The network dictionary has the following keys:
- 'network_name': text name of the network (by defualt accepts 'abm','here','here')
- 'links_fp': the filepath of the original links network data (must have a value)
- 'links_layer': if the file is a geopackage or geodatabase then use this to specify the layer to import
- 'nodes_fp': the filepath of the nodes data if available (replace with None if not available)
- 'nodes_layer': if the file is a geopackage or geodatabase then use this to specify the layer to import
- 'nodes_id': indicates the column name that contains the unique ids for each node (replace with None if not available)
- 'A': indicates the starting node id column name for the links (replace with None if not available)
- 'B': indicates the ending node id column name for the links (replace with None if not available)
- 'linkid': indicates the linkid if present (replace with None if not available)

## OpenStreetMap (OSM) Filtering
---

In [3]:
osm = {
       "network_name": 'osm',
       "edges_filepath": config['network_fp'] / 'osm.gpkg', 
       "edges_layer": 'edges', 
       "nodes_filepath": None, 
       "nodes_layer": None,
       "nodes_id": None,
       "A": "u",
       "B": "v",
       "linkid": 'linkid'
       }

In [4]:
#TODO running this function appears to create new linkids, need a consistent way of setting them, probably off a combination of osmid and geometry so that 
#I don't have to keep running the map matching code
from importlib import reload
reload(network_filter)
links, nodes = network_filter.filter_networks(settings,osm)

/Users/tannerpassmore/mambaforge/envs/bikewaysim/lib/python3.13/site-packages/geopandas/io/file.py:497: UserWarning: More than one layer found in 'osm.gpkg': 'edges' (default), 'raw', 'highway_nodes'. Specify layer parameter to avoid this warning.
  crs = pyogrio.read_info(path_or_bytes).get("crs")


Filtering the osm network.
no nodes layer
but links have reference ids
osm imported... took 0.08 minutes


In [5]:
#drop anything with highway == na
links = links[~links['highway'].isna()]

## Define link types

## Remove types
- Remove interstates and interstate ramps
- Remove sidewalks unless bicycles explicitly allowed
- Remove anything with access = "no"
- Remove any unpaved footways or paths surface = "dirt"

In [6]:
# #initialize an empty column to populate
links['link_type'] = None

## Conditions

In [7]:
#bicycle riding is allowed (no sometimees implies bike can still be walked)
bike_allowed = links['bicycle'].isin(['yes','permitted','permissive','designated'])
ped_allowed = links['foot'].isin(['yes','permitted','permissive','designated'])

## Paved Bike and Pedestrian Links (no cars allowed)


In [8]:
# sidewalks and crossings (helpful to identify these seperately so they can be removed)
sidewalks = links['footway'].isin(['sidewalk','crossing'])
links.loc[(bike_allowed==False) & sidewalks & links['link_type'].isna(),'link_type'] = 'sidewalk'

# ped only links that still provide additional connectivity
# we probably don't want these but they might be essential for connectivity
pedestrian = links['highway'].isin(['path','track','pedestrian','steps','corridor','footway'])
links.loc[(bike_allowed==False) & pedestrian & links['link_type'].isna(),'link_type'] = 'pedestrian'

# dirt trails/paths (surface tags aren't good for these, may need Garber's data)

# bike specific and ped links where cycling is explicictly tagged as allowed
conditions = (links["highway"]=='cycleway') | (pedestrian & bike_allowed)
links.loc[conditions & links['link_type'].isna() ,'link_type'] = 'bike'

## Unpaved Bike and Pedestrian Links (shortcuts, interim Beltline, etc.)

In [9]:
# if it's a bike or pedestrian link, create a new tag if the surface is not concrete/asphalt
# links['surface']

## Public and service roads

In [10]:
#parking lot aisles and driveways
parking_and_driveways = links['service'].isin(['parking_aisle','driveway'])
links.loc[parking_and_driveways & links['link_type'].isna(),'link_type'] = 'parking_and_driveways'

#service roads that aren't parking lots or driveways (and don't have names)
conditions = (links['highway'] == 'service') & (links['name'].isnull())
links.loc[conditions & links['link_type'].isna(),'link_type'] = 'service'

#find service links that still have a name
service_links_with_name = (links['highway'] == 'service') & (links['name'].isnull() == False)

#unclassified added 8/14/23 because there are several roads in Atlanta region marked this despite being public roads
filter_method = ['primary','primary_link','residential','secondary','secondary_link',
                    'tertiary','tertiary_link','trunk','trunk_link','unclassified','living_street'
                    'service','living_street'] 
filter_method = links["highway"].isin(filter_method)

#add back in service links with a name
conditions = osm_filter_method | service_links_with_name
links.loc[conditions & links["link_type"].isna(),'link_type'] = 'road'

## Restricted access (Interstates, highways, onramps)

In [11]:
#remove restricted access roads
restr_access = links['highway'].isin(['motorway','motorway_link'])
links.loc[restr_access & links['link_type'].isna(),'link_type'] = 'restricted_access_road'

## No bike

In [12]:
links.loc[links['bicycle']=='no','link_type'] = 'no_bike'

## Private/no access or closed road (explicilty doesn't allow bike/ped)
Had issues with this one removing key roads that would pass through Emory campus that are marked as access=private.

In [13]:


# no_access = (links['access'].isin(['no','private','permit'])) | (links['highway'] == 'disused')
# links[no_access & links['link_type'].isna()].explore()
# no_access = (links['access'].isin(['no','private','permit'])) | (links['highway'] == 'disused')
# #exception if bicycle/foot has a permissive/yes value
# #& (~bike_allowed | ~ped_allowed)
# links.loc[no_access,'link_type'] = 'no_access_or_private'

# #also give raceways this
# links.loc[links['highway']=='raceway','link_type'] = 'no_access_or_private'

## Construction / Proposed Links
Because this project is using old traces, the construction tag might remove features that were accessible previously. One example is McDonough near Lakewood or Jesse Hill Junior near Grady Memorial Hospital. There's is one feature that is under construction in the 2021 dataset that can be removed which is the PATH400 trail.

There are few sections of the Beltline that are marked as proposed.

In [14]:
links.loc[links['highway'].isin(['construction']),'link_type'] = 'road'
links.loc[links['highway'].isin(['proposed']),'link_type'] = 'no_access_or_private'

# put path400 into proposed as short term solution
links.loc[links['link_type'].isin(['construction']) & links['name'].isin(['PATH400']),'link_type'] = 'no_access_or_private'

In [15]:
# import ast
# construct = links.loc[links['highway'].isin(['construction','proposed'])].copy()
# construct['all_tags'] = construct['all_tags'].apply(lambda x: str({key:item for key,item in ast.literal_eval(x).items() if key != "@way_nodes"}))
# m = construct.explore()
# m.save(Path.home()/'Downloads/construction.html')

## NAs
Be sure to check if all links have been classified. Some OSM highway tags aren't clear and might need to be added on a case by case basis (e.g., track, construction, living_street).

In [16]:
links['link_type'].isna().sum()

np.int64(2)

## Reclassify sidewalks/crossings that connect bike links to roads
Sometimes seperated multi-use trails that are designated for bike travel are connected back to the road network using regular crosswalks or sidewalks. In this case, we want to re-classify those links as bike links as they'll break the connectivity of the trail in routing and map matching.

A prime example of this is the Stone Mountain Trail at both [Central Ave and Jackson St](https://www.openstreetmap.org/#map=19/33.761203/-84.376030)

In [18]:
#TODO make this a graph process where you can find connectors with X links or X feet

# connects to a bike link
bike_a = links.loc[links['link_type']=='bike','A'].tolist()
bike_b = links.loc[links['link_type']=='bike','B'].tolist()
bike_ab = set(bike_a+bike_b)

# connects to a road link
road_a = links.loc[links['link_type']=='road','A'].tolist()
road_b = links.loc[links['link_type']=='road','B'].tolist()
road_ab = set(road_a+road_b)

a_road_b_bike = (links['A'].isin(road_ab)) & (links['B'].isin(bike_ab))
b_bike_a_road = (links['A'].isin(bike_ab)) & (links['B'].isin(road_ab))

# length condition
# remove really long connections that are just sidewalks
length_cond = links['length_ft'] < 100

# links.loc[(links['link_type']=='sidewalk') & (a_road_b_bike | b_bike_a_road)].explore()
links.loc[(links['link_type']=='sidewalk') & length_cond & (a_road_b_bike | b_bike_a_road),'link_type'] = 'pedestrian'

## Specific reclassifications
Use this space to re-classify link types according to how they actually function and try to update the OSM versions.

In [19]:
# for now just classify them as pedestrian
cuthroughs = [391637440,909206331,391637443,391637444,395875912,1051679237,343041326,216277084,496238164]
links.loc[links['osmid'].isin(cuthroughs),'link_type'] = 'pedestrian'

## Make a boolean oneway column

In [20]:
links['oneway'] = links['oneway'] == 'yes'

## Export

In [21]:
network_filter.export(links,nodes,'osm',settings,osm,['osmid'])

Export took 0.01 minutes


In [22]:
# # FOR OTHER NETWORKS
# <!-- # HERE Filtering
# ---
# HERE is licensed data typically used for logistics.

# #### General Layer Filtering
# - Remove interstates and interstate ramps (controlled access = yes or ramp = yes) that don't allow bikes

# #### Road Layer Filtering
# - Keep all links that allow automobile access (ar_auto = yes)
# - Remove links with a speed limit < 6 mph (speed_cat = 8) as these are driveways

# #### Bike Layer Filtering
# - All links that do not allow auto access (ar_auto = no)

# #### Service Layer Filtering
# - All links that allow auto access with a speed limit < 6mph (ar_auto = y and speed_cat = 8)
# here = {
#        "network_name": 'here', # name for network
#        "edges_filepath": Path(config['here_fp']), #filepath for the links
#        "edges_layer":None, # layer name for links if gpkg or gdb
#        "nodes_filepath": None, # if there is not a nodes file put None
#        "nodes_layer": None, # layer name for nodes if gpkg or gdb (put none if no nodes)
#        "nodes_id": None, # column name of node id (put none if no node id)
#        "A": "REF_IN_ID", # starting node id reference column on links (put none if no reference links)
#        "B": "NREF_IN_ID", # starting node id reference column on links (put none if no reference links)
#        "linkid": 'LINK_ID'
#        }

# links, nodes = network_filter.filter_networks(settings,here)
# #initialize an empty column to populate
# links['link_type'] = np.nan
# #remove controlled access roads and ramps
# controlled_access = (links['CONTRACC'].str.contains('Y')) | (links['RAMP'].str.contains('Y'))
# links.loc[controlled_access & links['link_type'].isna(),'link_type'] = 'restricted_access_road'

# # road filters
# road_links = (links['AR_AUTO'].str.contains('Y')) & (links['SPEED_CAT'].str.contains('8') == False)
# links.loc[road_links & links['link_type'].isna(),'link_type'] = 'road'

# # bike filters
# bike_links = links['AR_AUTO'].str.contains('N')
# links.loc[bike_links & links['link_type'].isna(),'link_type'] = 'bike'

# # service filters
# service_links = (links['AR_AUTO'].str.contains('Y')) & (links['SPEED_CAT'].str.contains('8'))
# links.loc[service_links & links['link_type'].isna(),'link_type'] = 'service'

# network_filter.export(links,nodes,'here',settings)
# del links, nodes
# # ABM Filtering
# ---
# This is the modeling network used in the Atlanta Regional Commission's Activity Based Model. It was provided by the ARC and is available on request.

# #### General Layer Filtering
# - Remove all links except for principal arterials, minor arterials, collectors, and local roads (FACTYPE = 10, 11, or 14)

# #### Road Layer Filtering
# - The filtering done in the previous step was enough, no further filtering needed

# #### Bike Layer Filtering
# - No bike links present

# #### Service Layer Filtering
# - No service links present
# abm = {
#        "network_name": 'abm',
#        "edges_filepath": Path(config['abm_fp']),
#        "edges_layer":"DAILY_Link",
#        "nodes_filepath": Path(config['abm_fp']),
#        "nodes_layer":"DAILY_Node",
#        "nodes_id": "N",
#        "A": "A",
#        "B": "B",
#        "linkid": "A_B"
#        }

# links, nodes = network_filter.filter_networks(settings,abm)
# #need new way to identify these?
# #links = remove_directed_links(links,'abm')

# #explode and drop level to get rid of multi-index in abm layer
# links = links.explode().reset_index(drop=True)

# #remove interstates and centroid connectors
# abm_road = [10,11,14]
# controlled_access_and_centroids = links["FACTYPE"].isin(abm_road)
# links.loc[controlled_access_and_centroids,'link_type'] = 'centroids_and_access_links'

# network_filter.export(links,nodes,'abm',settings)
# del links, nodes -->
